## Replacing missing values
By the end of this lecture you will be able to:
- replace missing values with a constant
- replace missing values with a strategy

In [ ]:
import polars as pl

We create a simple `DataFrame` for this lecture

In [ ]:
df = pl.DataFrame(
    {
        "col1":[0,None,2,3],
        "col2":[0,None,None,3],
        "strings":["a",None,"c","d"]
    }
)
df

## Replace missing values with a constant
We replace missing values in an expression using `fill_null`

In [ ]:
(
    df
    .with_column(
        pl.all().fill_null(0).suffix("_new")
    )
)

Note that `fill_null` replaced `null` with a string `"0"` in `strings_new`

We can also replace the missing values with a string 

In [ ]:
(
    df
    .with_column(
        pl.all().fill_null("missing").suffix("_new")
    )
)

In this case `fill_null` has `cast` the columns from integer to string dtype!



## Replace missing values with a strategy
We can also replace missing values with a stragegy including:
- forward: replace with the previous non-`null` value
- backward: replace with the next non-`null` value
- min: replace with the smallest value in the `Series`
- max: replace with the largest value in the `Series`
- mean: replace with the median value in the `Series`
- zero: replace with `0`
- one: replace with `1`

### Forward strategy
In the forward strategy the missing values are replaced with the previous non-`null` values

In [ ]:
(
    df
    .with_columns(
        pl.all().fill_null(strategy="forward").suffix("_new")
    )
)

We can set a limit on how many rows to fill-forward or backward with `limit`

In [ ]:
(
    df
    .with_columns(
        pl.all().fill_null(strategy="forward",limit=1).suffix("_new")
    )
)

## Exercises
In the exercises you will develop your understanding of:
- replacing missing values with a constant
- replacing missing values with a strategy

### Exercise 1
Filter the `DataFrame` to have only the two rows with missing values in the `Embarked` column and then replace the missing values in the `Embarked` column with `"unknown"`

In [ ]:
csvFile = "../data/titanic.csv"
(
    pl.read_csv(csvFile)
   <blank>
)

### Exercise 2
Add a new column called `Age_filled` where missing values are replaced with the  value from the following row.

In [ ]:
csvFile = "../data/titanic.csv"
(
    pl.read_csv(csvFile)
    <blank>
    .select(["Age","Age_filled"])
)

Add three new columns called `Age_mean`, `Age_median` and `Age_interpolated` where missing values are replaced with the mean, median and interpolated values.

In [ ]:
csvFile = "../data/titanic.csv"
(
    pl.read_csv(csvFile)
    <blank>
    .select(["Age","Age_mean","Age_median","Age_interpolated"])
    .filter(pl.col("Age").is_null())
)

## Solutions

### Solution to Exercise 1
Filter the `DataFrame` to have only the two rows with missing values in the `Embarked` column and then replace the missing values in the `Embarked` column with `"unknown"`

In [ ]:
csvFile = "../data/titanic.csv"
(
    pl.read_csv(csvFile)
    .filter(
        pl.col("Embarked").is_null()
    )
    .with_column(
        pl.col("Embarked").fill_null("unknown")
    )
)

### Solution to Exercise 2
Add a new column called `Age_filled` where missing values are replaced with the  value from the following row.

In [ ]:
csvFile = "../data/titanic.csv"
(
    pl.read_csv(csvFile)
    .with_column(
        pl.col("Age").fill_null(strategy="backward").alias("Age_filled")
    )
    .select(["Age","Age_filled"])
)

Solution to Exercise 2 cont:

Add three new columns called `Age_mean`, `Age_median` and `Age_interpolated` where missing values are replaced with the mean, median and interpolated values.

In [ ]:
csvFile = "../data/titanic.csv"
(
    pl.read_csv(csvFile)
    .with_columns(
        [
            pl.col("Age").fill_null(strategy="mean").alias("Age_mean"),
            pl.col("Age").fill_null(pl.col("Age").median()).alias("Age_median"),
            pl.col("Age").interpolate().alias("Age_interpolated"),

        ]
            )
    .select(["Age","Age_mean","Age_median","Age_interpolated"])
    .filter(pl.col("Age").is_null())
)